## Instructions

### 1. Install several requirements

In [1]:
!pip3 install tensorflow
!pip3 install tensorboardX
!pip install transformers
!pip install spacy
!python -m spacy download en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 114.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 126.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 10.0 MB/s eta 0:00:00
     ━━━━

  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.6
    Uninstalling protobuf-4.21.6:
      Successfully uninstalled protobuf-4.21.6
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.5.3
    Uninstalling google-auth-oauthlib-0.5.3:
      Successfully uninstalled google-auth-oauthlib-0.5.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 54.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 129.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.0/757.0 kB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 60.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 130.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
2022-10-18 05:41:41.309138: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 05:41:41.520092: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors fro

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### 2. Place the BERT model-related files and GloVE word embeddings following the instructions below.

* 구글 드라이브 /code 에 있는 uncased_L-12_H-768_A-12.zip 과 glove.840B.300d.zip 을 './ABSC'에 업로드 후 아래 두 코드 실행

In [3]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [4]:
BERT_BASE_DIR='./uncased_L-12_H-768_A-12'

!python3 convert_tf_to_pytorch/convert_tf_checkpoint_to_pytorch.py \
  --tf_checkpoint_path $BERT_BASE_DIR/bert_model.ckpt \
  --bert_config_file $BERT_BASE_DIR/bert_config.json \
  --pytorch_dump_path $BERT_BASE_DIR/pytorch_model.bin

2022-06-15 06:38:36.730316: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Converting TensorFlow checkpoint from ./uncased_L-12_H-768_A-12/bert_model.ckpt
Loading bert/embeddings/LayerNorm/beta with shape [768]
Numpy array shape (768,)
Loading bert/embeddings/LayerNorm/gamma with shape [768]
Numpy array shape (768,)
Loading bert/embeddings/position_embeddings with shape [512, 768]
Numpy array shape (512, 768)
Loading bert/embeddings/token_type_embeddings with shape [2, 768]
Numpy array shape (2, 768)
Loading bert/embeddings/word_embeddings with shape [30522, 768]
Numpy array shape (30522, 768)
Loading bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_0/attention/output/LayerNorm/gamma with shape [768

Numpy array shape (768, 3072)
Loading bert/encoder/layer_2/output/LayerNorm/beta with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_2/output/LayerNorm/gamma with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_2/output/dense/bias with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_2/output/dense/kernel with shape [3072, 768]
Numpy array shape (3072, 768)
Loading bert/encoder/layer_3/attention/output/LayerNorm/beta with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_3/attention/output/LayerNorm/gamma with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_3/attention/output/dense/bias with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_3/attention/output/dense/kernel with shape [768, 768]
Numpy array shape (768, 768)
Loading bert/encoder/layer_3/attention/self/key/bias with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_3/attention/self/key/kernel with shape [768, 768]
Nump

Numpy array shape (3072, 768)
Loading bert/pooler/dense/bias with shape [768]
Numpy array shape (768,)
Loading bert/pooler/dense/kernel with shape [768, 768]
Numpy array shape (768, 768)
Loading cls/predictions/output_bias with shape [30522]
Numpy array shape (30522,)
Loading cls/predictions/transform/LayerNorm/beta with shape [768]
Numpy array shape (768,)
Loading cls/predictions/transform/LayerNorm/gamma with shape [768]
Numpy array shape (768,)
Loading cls/predictions/transform/dense/bias with shape [768]
Numpy array shape (768,)
Loading cls/predictions/transform/dense/kernel with shape [768, 768]
Numpy array shape (768, 768)
Loading cls/seq_relationship/output_bias with shape [2]
Numpy array shape (2,)
Loading cls/seq_relationship/output_weights with shape [2, 768]
Numpy array shape (2, 768)
Loading embeddings/LayerNorm/beta
Loading embeddings/LayerNorm/gamma
Loading embeddings/position_embeddings
Loading embeddings/token_type_embeddings
Loading embeddings/word_embeddings
Loading e

In [10]:
!unzip './glove.840B.300d.zip' -d './datasets/glove'

Archive:  ./glove.840B.300d.zip
  inflating: ./datasets/glove/glove.840B.300d.txt  


### 3. Create dependency graphs by running following codes

* final output files are f'./{dataset_name}_datas.pkl'

In [8]:
!python generateGraph_spacy.py

2022-06-15 08:29:09.633270: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
100%|█████████████████████████████████████████| 638/638 [00:22<00:00, 28.20it/s]


In [11]:
!python data_utils.py

2022-06-15 08:47:40.796772: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
preparing twitter dataset ...
load successfully
loading word vectors ...
building embedding_matrix: 300_twitter_embedding_matrix.pkl
100%|█████████████████████████████████████████| 692/692 [00:57<00:00, 11.94it/s]
preparing rest14 dataset ...
load successfully
loading word vectors ...
building embedding_matrix: 300_rest14_embedding_matrix.pkl
100%|███████████████████████████████████████| 1120/1120 [01:31<00:00, 12.29it/s]
preparing lap14 dataset ...
load successfully
loading word vectors ...
building embedding_matrix: 300_lap14_embedding_matrix.pkl
100%|█████████████████████████████████████████| 638/638 [00:51<00:00, 12.51it/s]


In [6]:
import random
random.seed(1)
for i in range(10):
    var_name = 'seed'+str(i+1)
    locals()[var_name] = random.randint(0,10000000)

In [48]:
!python run_classifier_word_roberta.py \
--model_name roberta_gcls \
--task_name=laptop \
--graph_type 'dg' \
--L_config_base '0,0,0,0,1,1,1,1,2,2,2,2' \
--g_config '1,1,1,1' \
--g_pooler 'att' \
--data_dir=/home/ikhyuncho23/GoBERTa/datasets/semeval14/laptops/ \
--vocab_file=/home/ikhyuncho23/GoBERTa/roberta_files/vocab.json \
--roberta_config_file=/home/ikhyuncho23/GoBERTa/roberta_files/config.json \
--max_seq_length=128 \
--train_batch_size 32 \
--eval_batch_size 32 \
--learning_rate 1.5e-5 \
--num_train_epochs 20.0 \
--local_rank -1 \
--gpu_id 0 \
--model_save_path trained_model_ckpts/ \
--para_LSR 0.2 \
--log_step 5 \
--seed $seed2

2022-10-18 08:12:03.140742: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 08:12:03.350471: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-18 08:12:03.391567: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-18 08:12:04.216072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Training:   5%|█▊                                | 4/73 [00:01<00:18,  3.75it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.01it/s]
Results: train_acc: 0.350000 | train_f1: 0.329541 | train_loss: 1.102278 | eval_accuracy: 0.534483 | eval_loss: 1.079653 | eval_f1: 0.232210 | max_test_acc: 0.534483 | max_test_f1: 0.232210
Training:  12%|████▏                             | 9/73 [00:03<00:16,  3.89it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.06it/s]
Results: train_acc: 0.381250 | train_f1: 0.304717 | train_loss: 0.645680 | eval_accuracy: 0.517241 | eval_loss: 1.076150 | eval_f1: 0.244643 | max_test_acc: 0.534483 | max_test_f1: 0.244643
Training:  19%|██████▎                          | 14/73 [00:05<00:15,  3.90it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.09it/s]
Results: train_acc: 0.389583 | train_f1: 0.321987 | train_loss: 0.384086 | eval_accuracy: 0.206

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.07it/s]
Results: train_acc: 0.416667 | train_f1: 0.310234 | train_loss: 0.119068 | eval_accuracy: 0.217868 | eval_loss: 1.123245 | eval_f1: 0.138443 | max_test_acc: 0.540752 | max_test_f1: 0.332628
Training:  67%|██████████████████████▏          | 49/73 [00:18<00:06,  3.91it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.09it/s]
Results: train_acc: 0.417500 | train_f1: 0.323484 | train_loss: 0.106441 | eval_accuracy: 0.214734 | eval_loss: 1.147261 | eval_f1: 0.132269 | max_test_acc: 0.540752 | max_test_f1: 0.332628
Training:  74%|████████████████████████▍        | 54/73 [00:19<00:04,  3.91it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.06it/s]
Results: train_acc: 0.418182 | train_f1: 0.326599 | train_loss: 0.098103 | eval_accuracy: 0.463950 | eval_loss: 1.074314 | eval_f1: 0.328352 | max_test_acc: 0.540752 | max_test_f1: 0.332628

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.94it/s]
Results: train_acc: 0.429688 | train_f1: 0.321359 | train_loss: 0.536927 | eval_accuracy: 0.534483 | eval_loss: 1.061423 | eval_f1: 0.232210 | max_test_acc: 0.540752 | max_test_f1: 0.354375
Training:  22%|███████▏                         | 16/73 [00:05<00:14,  3.90it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.97it/s]
Results: train_acc: 0.444853 | train_f1: 0.318098 | train_loss: 0.329961 | eval_accuracy: 0.532915 | eval_loss: 1.052544 | eval_f1: 0.237246 | max_test_acc: 0.540752 | max_test_f1: 0.354375
Training:  29%|█████████▍                       | 21/73 [00:07<00:13,  3.89it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.97it/s]
Results: train_acc: 0.447443 | train_f1: 0.311529 | train_loss: 0.251749 | eval_accuracy: 0.529781 | eval_loss: 1.057354 | eval_f1: 0.347984 | max_test_acc: 0.540752 | max_test_f1: 0.354375

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.86it/s]
Results: train_acc: 0.448803 | train_f1: 0.320178 | train_loss: 0.113295 | eval_accuracy: 0.534483 | eval_loss: 1.039641 | eval_f1: 0.232210 | max_test_acc: 0.567398 | max_test_f1: 0.388079
Training:  70%|███████████████████████          | 51/73 [00:18<00:05,  3.89it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.95it/s]
Results: train_acc: 0.448317 | train_f1: 0.316047 | train_loss: 0.098240 | eval_accuracy: 0.498433 | eval_loss: 1.078267 | eval_f1: 0.367210 | max_test_acc: 0.567398 | max_test_f1: 0.388079
Training:  77%|█████████████████████████▎       | 56/73 [00:20<00:04,  3.86it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.91it/s]
Results: train_acc: 0.451206 | train_f1: 0.320860 | train_loss: 0.093424 | eval_accuracy: 0.490596 | eval_loss: 1.069235 | eval_f1: 0.367479 | max_test_acc: 0.567398 | max_test_f1: 0.388079

...

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.97it/s]
Results: train_acc: 0.993644 | train_f1: 0.992893 | train_loss: 0.002750 | eval_accuracy: 0.805643 | eval_loss: 0.911762 | eval_f1: 0.768686 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  86%|████████████████████████████▍    | 63/73 [00:22<00:02,  3.89it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.97it/s]
Results: train_acc: 0.994141 | train_f1: 0.993409 | train_loss: 0.000495 | eval_accuracy: 0.794671 | eval_loss: 1.132714 | eval_f1: 0.743884 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  93%|██████████████████████████████▋  | 68/73 [00:24<00:01,  3.88it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.97it/s]
Results: train_acc: 0.993207 | train_f1: 0.992212 | train_loss: 0.004108 | eval_accuracy: 0.797806 | eval_loss: 1.015443 | eval_f1: 0.752944 | max_test_acc: 0.829154 | max_test_f1: 0.799676

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.87it/s]
Results: train_acc: 0.989583 | train_f1: 0.986851 | train_loss: 0.004563 | eval_accuracy: 0.774295 | eval_loss: 1.250692 | eval_f1: 0.714051 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  34%|███████████▎                     | 25/73 [00:09<00:12,  3.88it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.94it/s]
Results: train_acc: 0.990385 | train_f1: 0.987969 | train_loss: 0.007051 | eval_accuracy: 0.789969 | eval_loss: 1.104289 | eval_f1: 0.743531 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  41%|█████████████▌                   | 30/73 [00:11<00:11,  3.88it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.02it/s]
Results: train_acc: 0.991935 | train_f1: 0.989879 | train_loss: 0.000716 | eval_accuracy: 0.793103 | eval_loss: 1.018688 | eval_f1: 0.751946 | max_test_acc: 0.829154 | max_test_f1: 0.799676

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.03it/s]
Results: train_acc: 0.992746 | train_f1: 0.991598 | train_loss: 0.001495 | eval_accuracy: 0.816614 | eval_loss: 0.969746 | eval_f1: 0.780830 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  82%|███████████████████████████      | 60/73 [00:22<00:03,  3.90it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.01it/s]
Results: train_acc: 0.992828 | train_f1: 0.991645 | train_loss: 0.001284 | eval_accuracy: 0.810345 | eval_loss: 1.024730 | eval_f1: 0.771876 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  89%|█████████████████████████████▍   | 65/73 [00:24<00:02,  3.90it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.03it/s]
Results: train_acc: 0.992898 | train_f1: 0.991674 | train_loss: 0.000938 | eval_accuracy: 0.800940 | eval_loss: 1.100650 | eval_f1: 0.757853 | max_test_acc: 0.829154 | max_test_f1: 0.799676

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 20.00it/s]
Results: train_acc: 0.998264 | train_f1: 0.997705 | train_loss: 0.002487 | eval_accuracy: 0.783699 | eval_loss: 1.176917 | eval_f1: 0.734506 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  30%|█████████▉                       | 22/73 [00:07<00:13,  3.90it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.93it/s]
Results: train_acc: 0.998641 | train_f1: 0.998205 | train_loss: 0.001664 | eval_accuracy: 0.794671 | eval_loss: 1.115725 | eval_f1: 0.751728 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  37%|████████████▏                    | 27/73 [00:09<00:11,  3.90it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.04it/s]
Results: train_acc: 0.997768 | train_f1: 0.997090 | train_loss: 0.003527 | eval_accuracy: 0.796238 | eval_loss: 1.092649 | eval_f1: 0.753168 | max_test_acc: 0.829154 | max_test_f1: 0.799676

Training:  78%|█████████████████████████▊       | 57/73 [00:20<00:04,  3.90it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.01it/s]
Results: train_acc: 0.996767 | train_f1: 0.995843 | train_loss: 0.001527 | eval_accuracy: 0.810345 | eval_loss: 1.004635 | eval_f1: 0.772242 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  85%|████████████████████████████     | 62/73 [00:22<00:02,  3.89it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:00<00:00, 20.02it/s]
Results: train_acc: 0.996032 | train_f1: 0.995174 | train_loss: 0.001707 | eval_accuracy: 0.786834 | eval_loss: 1.027662 | eval_f1: 0.740131 | max_test_acc: 0.829154 | max_test_f1: 0.799676
Training:  92%|██████████████████████████████▎  | 67/73 [00:24<00:01,  3.89it/s]lr:  1.5e-05

Evaluating: 100%|███████████████████████████████| 20/20 [00:01<00:00, 19.99it/s]
Results: train_acc: 0.995864 | train_f1: 0.994964 | train_loss: 0.004011 | eval_accuracy: 0.761